In [2]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.svm import OneClassSVM

# Load the CSV file
data = pd.read_csv('/kaggle/input/gait-analysis/gait_features_cnn_trained.csv')

# Split the data into training and testing sets, while maintaining the ratio of labels in both sets
train, test = train_test_split(data, test_size=0.2, stratify=data['label'])

# Save the training and testing sets to CSV files
train.to_csv('train.csv', index=False)
test.to_csv('test.csv', index=False)

# Load the CSV file containing the anomalies
df_anomalies = pd.read_csv('/kaggle/input/gait-analysis/gait_features_cnn_nottrained.csv')

# Split the anomalies into training and testing sets, while maintaining the ratio of labels in both sets
train_anomalies, test_anomalies = train_test_split(df_anomalies, test_size=0.2, stratify=df_anomalies['label'])

# Save the training and testing sets to CSV files
train_anomalies.to_csv('train_anomalies.csv', index=False)
test_anomalies.to_csv('test_anomalies.csv', index=False)

# Split features and labels for trained data
X_trained = data.iloc[:, :-1]
y_trained = pd.Series([1] * len(data)) # Set labels to 1 (trained)

# Split features and labels for anomalies data
X_anomalies_train = train_anomalies.iloc[:, :-1]
y_anomalies_train = pd.Series([-1] * len(train_anomalies)) # Set labels to -1 (anomalies)

X_anomalies_test = test_anomalies.iloc[:, :-1]
y_anomalies_test = pd.Series([-1] * len(test_anomalies)) # Set labels to -1 (anomalies)

# Scale the feature values
scaler = StandardScaler()

X_trained_scaled = scaler.fit_transform(X_trained)
X_anomalies_train_scaled = scaler.transform(X_anomalies_train)
X_anomalies_test_scaled = scaler.transform(X_anomalies_test)

# Train the One-class SVM model on the trained data and the anomalies
ocsvm = OneClassSVM(kernel='rbf', nu=0.1, gamma=0.1)
ocsvm.fit(X_trained_scaled)

# Evaluate the model on the test set containing trained data and anomalies
y_test = pd.concat([pd.Series([1] * len(test)), y_anomalies_test], ignore_index=True)
X_test_scaled = scaler.transform(pd.concat([test.iloc[:, :-1], X_anomalies_test]))

y_pred = ocsvm.predict(X_test_scaled)

# Convert the predictions to binary labels
y_pred_binary = (y_pred >= 0.)

# Print the classification report
print(classification_report(y_test, y_pred_binary))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


              precision    recall  f1-score   support

           0       0.99      0.89      0.94      2477
           1       0.00      0.00      0.00        22

    accuracy                           0.88      2499
   macro avg       0.50      0.45      0.47      2499
weighted avg       0.98      0.88      0.93      2499

              precision    recall  f1-score   support

           0       1.00      0.95      0.97       220
           1       0.00      0.00      0.00         0

    accuracy                           0.95       220
   macro avg       0.50      0.47      0.49       220
weighted avg       1.00      0.95      0.97       220



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:88: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWar